In [9]:
import os
import csv
import pandas as pd
import glob

In [10]:
directory = r'R:\RawData\Elite Star\React Stores\Client Download Folders'

In [11]:
def delete_all_except_recent(directory):
    # Get a list of all files in the directory and its subdirectories
    files = []
    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))

    # Sort the files by modification time in descending order
    files.sort(key=os.path.getmtime, reverse=True)

    # Keep track of the most recent file in each directory
    most_recent_files = {}

    # Keep track of the deleted files
    deleted_files = []

    # Keep the most recent file in each directory and delete the rest
    for file in files:
        # Get the directory of the file
        directory = os.path.dirname(file)

        # If there is only one file in the directory, skip it
        if len(glob.glob(os.path.join(directory, "*"))) == 1:
            continue

        # If the directory is not in the most_recent_files dictionary, add it
        if directory not in most_recent_files:
            most_recent_files[directory] = file
        else:
            # If the file is older than the most recent file in the directory, delete it
            if os.path.getmtime(file) < os.path.getmtime(most_recent_files[directory]):
                os.remove(file)
                deleted_files.append(os.path.basename(file))
            else:
                # Otherwise, update the most recent file in the directory
                os.remove(most_recent_files[directory])
                deleted_files.append(os.path.basename(most_recent_files[directory]))
                most_recent_files[directory] = file

    # Keep the most recent file in each directory
    for directory, most_recent_file in most_recent_files.items():
        files_in_directory = glob.glob(os.path.join(directory, "*"))
        for file in files_in_directory:
            if file != most_recent_file:
                os.remove(file)
                deleted_files.append(os.path.basename(file))

    # Create a Pandas DataFrame of the deleted files
    df = pd.DataFrame(deleted_files, columns=["Deleted Files"])
    df["Deleted Files"] = df["Deleted Files"].apply(lambda x: os.path.basename(x))

    return df

In [12]:
files_df = delete_all_except_recent(directory)

In [13]:
files_df

,Deleted Files
0,SaleAudit-2023-05-08.csv
1,SaleAudit_Premjee TZN_2023-05-08.csv
2,SaleAudit_Northam_2023-05-08.csv
3,SaleAudit_Mogwase DC_2023-05-08.csv
4,SaleAudit_Premjee LT_2023-05-08.csv
5,SaleAudit_A1 SUPERMARKET_2023-05-08.csv
6,SaleAudit_Mogwase Grocery_2023-05-08.csv
7,SaleAudit_Zeerust_2023-05-08.csv
8,SaleAudit_Potchefstroom_2023-05-08.csv
9,SaleAudit_Mogwase DC_2023-05-01.csv


In [14]:
def delete_rows_starting_with(directory, character):
    deleted_rows = []
    modified_files = []

    # Walk through the directory and its subdirectories
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                # If the file is a CSV file, open it and read its rows
                filename = os.path.join(root, file)
                with open(filename, "r") as f:
                    reader = csv.reader(f)
                    rows = list(reader)

                # Delete any row that starts with the given character
                deleted = [row for row in rows if row[0].startswith(character)]
                rows = [row for row in rows if not row[0].startswith(character)]

                # If any rows were deleted, add them to the list of deleted rows
                if deleted:
                    for row in deleted:
                        deleted_rows.append([os.path.basename(filename), row])

                    modified_files.append(os.path.basename(filename))

                # Write the updated rows back to the CSV file
                with open(filename, "w", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerows(rows)

    # Convert the deleted rows and modified files to a pandas dataframe
    df = pd.DataFrame(deleted_rows, columns=["Filename", "Rows Deleted"])
    df["Modified"] = df["Filename"].isin(modified_files)

    return df


In [15]:
df = delete_rows_starting_with(directory, "(")

In [ ]:
df